In [1]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
import pickle
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler 
from itertools import *

In [2]:
train = pd.read_csv("./trainning_data/label_risk_company.csv")

In [3]:
test = pd.read_csv("./test_data/test_data.csv")

In [4]:
base_info = pd.read_csv("./trainning_data/revised_attribute/company_base_info_new.csv")

In [5]:
base_info.to_csv("./trainning_data/revised_attribute/company_base_info_new.csv",index=None)

In [6]:
base_info.head()

,entid,web_dum,ENTTYPE,INDUSTRYPHY,REGCAP,REGCAPCUR,RECCAP,PARNUM,LIMPARNUM,EXENUM,cancel_dum,certificate_num,tax_dum
0,122697049,1,10,9,8103057.4,156.0,4700000.0,0.0,0.0,0.0,0,4.0,0
1,959830672,1,5,11,38000.0,156.0,1000.0,0.0,0.0,0.0,0,2.0,0
2,564421968,0,8,12,5000.0,156.0,5000.0,0.0,0.0,0.0,1,0.0,0
3,620868429,0,8,9,10000.0,156.0,10000.0,0.0,0.0,0.0,0,0.0,0
4,1176966297,0,55,11,3000.0,156.0,3000.0,0.0,0.0,0.0,0,0.0,0


In [7]:
with open("./tax_year_features_1.pkl", 'rb') as fi:
    tax_year_features = pickle.load(fi)
with open("./base_info_type_features_0.pkl", 'rb') as fi:
    base_info_type_features_features = pickle.load(fi)
with open("./financial_features_0.pkl", 'rb') as fi:
    financial_features = pickle.load(fi)
with open("./tax_abnormal_feature_0.pkl", 'rb') as fi:
    tax_abnormal_features = pickle.load(fi)
with open("./socia_fee_features_0.pkl", 'rb') as fi:
    socia_fee_features = pickle.load(fi)
with open("./tax_asset_features_1.pkl", 'rb') as fi:
    tax_asset_features = pickle.load(fi) 
with open("./isna_features_0.pkl", 'rb') as fi:
    isna_features = pickle.load(fi) 
        

In [8]:
tax_abnormal_features.head()

,tax_type,tax_state,ttype,province
0,-1,-1,-1,-1
1,-1,-1,-1,-1
2,-1,-1,-1,-1
3,-1,-1,-1,-1
4,-1,-1,-1,-1


In [9]:
base_info = pd.concat([base_info,
                       tax_year_features,
                       base_info_type_features_features,
                       financial_features,
                       tax_abnormal_features,
                       socia_fee_features
                       ,tax_asset_features
                       ,isna_features
                      ],axis = 1)

In [10]:
tax_asset_features.columns

Index(['gross_income', 'debt_income_rate', 'other_gross_rate',
       'main_gross_rate', 'yea_sub_exsit', 'adjust_gross_income'],
      dtype='object')

In [11]:
# auto_bin_features = ["REGCAP","RECCAP","PARNUM","LIMPARNUM","EXENUM","certificate_num"]
auto_bin_features = ["REGCAP","RECCAP","PARNUM","LIMPARNUM","EXENUM","certificate_num",
                     "profit_loss","profit_loss_history_percent","profit_loss_asset",
                    "revenue","revenue_history_percent","revenue_asset",
                    "sales_cost","sales_cost_history_percent","sales_cost_asset",
                    "sales_tax","sales_tax_history_percent","sales_tax_asset",
                    "sales_expense","sales_expense_history_percent","sales_expense_asset",
                    "G_expense","G_expense_history_percent","G_expense_asset",
                    "finance_expense","finance_expense_history_percent","finance_expense_asset",
                    "asset_devaluation_losses","asset_devaluation_losses_history_percent","asset_devaluation_losses_asset",
                    "fair_value_change","fair_value_change_history_percent","fair_value_change_asset",
                    "investment_income","investment_income_history_percent","investment_income_asset",
                    "operation_profit","operation_profit_history_percent","operation_profit_asset",
                    "non_operating_income","non_operating_income_history_percent","non_operating_income_asset",
                    "non_operating_expense","non_operating_expense_history_percent","non_operating_expense_asset",
                    "total_profit","total_profit_history_percent","total_profit_asset",
                    "assets","assets_history_percent","assets_asset",
                    "assets_re","assets_re_history_percent","assets_re_asset",
                    "payrol_expense","payrol_expense_history_percent","payrol_expense_asset",
                    "main_business_income","main_business_income_history_percent","main_business_income_asset",
                    "main_business_cost","main_business_cost_history_percent","main_business_cost_asset",
                    "other_business_income","other_business_income_history_percent","other_business_income_asset",
                    "other_operating_cost","other_operating_cost_history_percent","other_operating_cost_asset",
                    "retained_profits","retained_profits_history_percent","retained_profits_asset",
                    "gross_profit","gross_profit_history_percent","gross_profit_asset",
                    "main_business_gross","main_business_gross_history_percent","main_business_gross_asset",
                    "income_tax_adjuste","income_tax_adjuste_history_percent","income_tax_adjuste_asset",
                    "income_taxable","income_taxable_history_percent","income_taxable_asset",
                    "incometax_taxable","incometax_taxable_history_percent","incometax_taxable_asset",
                    "incometax_amount","incometax_amount_history_percent","incometax_amount_asset",
                    "incometax_actual","incometax_actual_history_percent","incometax_actual_asset",
                     "INDUSTRYPHY_num","ENTTYPE_num",
                     'A1_Interest_Coverage_Multiple_A', 'A2_Interest_overage_Multiple B',
                     'A3_Asset_liability_ratio', 'A4_Ratio_of_equity_to_debt', 'A5_EBITDA',
                     'B1_equity_ratio', 'B2_Profit_ratio_of_main_business',
                     'B3_Profit_from_financial_activities', 'B4_Operating_profit_ratio',
                     'B5_Flow_rate', 'C1_ROA', 'C2_ROE', 'C3_gross_margin',
                     'C4_Expense_ratio_during_sales',
                     'C5_Operating_margin_before_interest_and_tax', 'D1_Financial_leverage',
                     'D2_EM', 'D3_ER', 'E1_Capital_preservation_and_appreciation_rate',
                     'E2_Growth_rate_of_total_assets', 'E3_Growth_rate_of_revenue',
                     "so1_history_percent",
                    "so2_history_percent",
                    "so3_history_percent",
                    "so4_history_percent",
                    "so5_history_percent",
                    "totalWagesSo1_history_percent",
                    "totalWagesSo2_history_percent",
                    "totalWagesSo3_history_percent",
                    "totalWagesSo5_history_percent",
                    "totalPaymentSo1_history_percent",
                    "totalPaymentSo2_history_percent",
                    "totalPaymentSo3_history_percent",
                    "totalPaymentSo4_history_percent",
                    "totalPaymentSo5_history_percent",
                    "unPaidSocialInsSo1_history_percent",
                    "unPaidSocialInsSo2_history_percent",
                    "unPaidSocialInsSo3_history_percent",
                    "unPaidSocialInsSo4_history_percent",
                    "unPaidSocialInsSo5_history_percent"
                    , 'gross_income', 'debt_income_rate', 'other_gross_rate',
                    'main_gross_rate', 'adjust_gross_income'
                    ]

In [12]:
bin_features_head =  [ x + "_bin" for x in auto_bin_features]

In [13]:
categrical_features = ["web_dum","ENTTYPE","INDUSTRYPHY","cancel_dum","tax_dum",
                       "profit_loss_history_more","profit_loss_empty",
                        "revenue_history_more","revenue_empty",
                        "sales_cost_history_more","sales_cost_empty",
                        "sales_tax_history_more","sales_tax_empty",
                        "sales_expense_history_more","sales_expense_empty",
                        "G_expense_history_more","G_expense_empty",
                        "finance_expense_history_more","finance_expense_empty",
                        "asset_devaluation_losses_history_more","asset_devaluation_losses_empty",
                        "fair_value_change_history_more","fair_value_change_empty",
                        "investment_income_history_more","investment_income_empty",
                        "operation_profit_history_more","operation_profit_empty",
                        "non_operating_income_history_more","non_operating_income_empty",
                        "non_operating_expense_history_more","non_operating_expense_empty",
                        "total_profit_history_more","total_profit_empty",
                        "assets_history_more","assets_empty",
                        "assets_re_history_more","assets_re_empty",
                        "payrol_expense_history_more","payrol_expense_empty",
                        "main_business_income_history_more","main_business_income_empty",
                        "main_business_cost_history_more","main_business_cost_empty",
                        "other_business_income_history_more","other_business_income_empty",
                        "other_operating_cost_history_more","other_operating_cost_empty",
                        "retained_profits_history_more","retained_profits_empty",
                        "gross_profit_history_more","gross_profit_empty",
                        "main_business_gross_history_more","main_business_gross_empty",
                        "income_tax_adjuste_history_more","income_tax_adjuste_empty",
                        "income_taxable_history_more","income_taxable_empty",
                        "incometax_taxable_history_more","incometax_taxable_empty",
                        "incometax_amount_history_more","incometax_amount_empty",
                        "incometax_actual_history_more","incometax_actual_empty",
                        "INDUSTRYPHY_risk_exist","INDUSTRYPHY_capital_higher","INDUSTRYPHY_paidin_capital_higher",
                        "ENTTYPE_risk_exist","ENTTYPE_capital_higher","ENTTYPE_paidin_capital_higher",
                        'tax_type', 'tax_state', 'ttype', 'province',
                       "so1_history_more","so1_empty",
                        "so2_history_more","so2_empty",
                        "so3_history_more","so3_empty",
                        "so4_history_more","so4_empty",
                        "so5_history_more","so5_empty",
                        "totalWagesSo1_history_more","totalWagesSo1_empty",
                        "totalWagesSo2_history_more","totalWagesSo2_empty",
                        "totalWagesSo3_history_more","totalWagesSo3_empty",
                        "totalWagesSo5_history_more","totalWagesSo5_empty",
                        "totalPaymentSo1_history_more","totalPaymentSo1_empty",
                        "totalPaymentSo2_history_more","totalPaymentSo2_empty",
                        "totalPaymentSo3_history_more","totalPaymentSo3_empty",
                        "totalPaymentSo4_history_more","totalPaymentSo4_empty",
                        "totalPaymentSo5_history_more","totalPaymentSo5_empty",
                        "unPaidSocialInsSo1_history_more","unPaidSocialInsSo1_empty",
                        "unPaidSocialInsSo2_history_more","unPaidSocialInsSo2_empty",
                        "unPaidSocialInsSo3_history_more","unPaidSocialInsSo3_empty",
                        "unPaidSocialInsSo4_history_more","unPaidSocialInsSo4_empty",
                        "unPaidSocialInsSo5_history_more","unPaidSocialInsSo5_empty",
                         'ispublic_empty', 'yea_sub_exsit',
                       '1_ispublic_isna',
                         '1_ANCHEYEAR_isna',
                         '1_so1_isna',
                         '1_so2_isna',
                         '1_so3_isna',
                         '1_so4_isna',
                         '1_so5_isna',
                         '1_totalWagesSo1_isna',
                         '1_totalWagesSo2_isna',
                         '1_totalWagesSo3_isna',
                         '1_totalWagesSo5_isna',
                         '1_totalPaymentSo1_isna',
                         '1_totalPaymentSo2_isna',
                         '1_totalPaymentSo3_isna',
                         '1_totalPaymentSo4_isna',
                         '1_totalPaymentSo5_isna',
                         '1_unPaidSocialInsSo1_isna',
                         '1_unPaidSocialInsSo2_isna',
                         '1_unPaidSocialInsSo3_isna',
                         '1_unPaidSocialInsSo4_isna',
                         '1_unPaidSocialInsSo5_isna',
                         '2_ALTDATE_isna',
                         '2_ALTTIME_isna',
                         '3_ANCHEYEAR_isna',
                         '3_INV_isna',
                         '3_TRANSAMPR_isna',
                         '3_TRANSAMAFT_isna',
                         '3_ALTDATE_isna',
                         '4_NAME_isna',
                         '4_indate_isna',
                         '4_outdate_isna',
                         '5_honor_num_isna',
                         '5_illegad_num_isna',
                         '5_yea_sub_isna',
                         '5_web_dum_isna',
                         '5_ENTTYPE_isna',
                         '5_INDUSTRYPHY_isna',
                         '5_REGCAP_isna',
                         '5_REGCAPCUR_isna',
                         '5_RECCAP_isna',
                         '5_PARNUM_isna',
                         '5_LIMPARNUM_isna',
                         '5_PARFORM_isna',
                         '5_EXENUM_isna',
                         '5_brand_num_isna',
                         '5_cancel_dum_isna',
                         '5_certificate_num_isna',
                         '5_tax_dum_isna',
                         '6_ANCHEYEAR_isna',
                         '6_ispublic_isna',
                         '6_ASSGRO_isna',
                         '6_LIAGRO_isna',
                         '6_VENDINC_isna',
                         '6_MAIBUSINC_isna',
                         '6_PROGRO_isna',
                         '6_NETINC_isna',
                         '6_RATGRO_isna',
                         '6_TOTEQU_isna',
                         '7_ANCHEYEAR_isna',
                         '7_ANCHEDATE_isna',
                         '7_EMPNUM_isna',
                         '7_WOMEMPNUM_isna',
                         '7_BUSST_isna',
                         '8_region_id_isna',
                         '8_entertype_tax_isna',
                         '8_industry_tax_isna',
                         '8_registertype_isna',
                         '8_economic_type_isna',
                         '8_incometax_rate_isna',
                         '8_collection_type_isna',
                         '8_EMPNUM_isna',
                         '8_year_isna',
                         '9_taxtype_isna',
                         '9_debt_isna',
                         '10_tax_type_isna',
                         '10_tax_state_isna',
                         '10_cdate_isna',
                         '10_ttype_isna',
                         '10_province_isna',
                         '11_entid_inv_isna',
                         '11_CONAM_before_isna',
                         '11_CONAM_after_isna',
                         '11_CONPROP_before_isna',
                         '11_CONPROP_after_isna',
                         '11_ttype_isna',
                         '11_ATDATE_isna',
                         '12_MEMNNUM_isna',
                         '12_FARNUM_isna',
                         '12_ANNNEWMEM_isna',
                         '12_ANNREDMEM_isna',
                         '12_ANCHEYEAR_isna',
                         '13_year_isna',
                         '13_islatest_isna',
                         '13_profit_loss_isna',
                         '13_revenue_isna',
                         '13_sales_cost_isna',
                         '13_sales_tax_isna',
                         '13_sales_expense_isna',
                         '13_G_expense_isna',
                         '13_finance_expense_isna',
                         '13_asset_devaluation_losses_isna',
                         '13_fair_value_change_isna',
                         '13_investment_income_isna',
                         '13_operation_profit_isna',
                         '13_non_operating_income_isna',
                         '13_non_operating_expense_isna',
                         '13_total_profit_isna',
                         '13_assets_isna',
                         '13_assets_re_isna',
                         '13_payrol_expense_isna',
                         '13_welfare_expenses_isna',
                         '13_education_expenses_isna',
                         '13_union_funds_isna',
                         '13_ad_expenses_isna',
                         '13_entertainment_expenses_isna',
                         '13_assets_loss_isna',
                         '13_asset_devaluation_allowance_isna',
                         '13_main_business_income_isna',
                         '13_main_business_cost_isna',
                         '13_other_business_income_isna',
                         '13_other_operating_cost_isna',
                         '13_retained_profits_isna',
                         '13_gross_profit_isna',
                         '13_main_business_gross_isna',
                         '13_income_tax_adjuste_isna',
                         '13_income_taxable_isna',
                         '13_incometax_taxable_isna',
                         '13_incometax_amount_isna',
                         '13_incometax_actual_isna'
                      ]

In [14]:
# 分箱
Discretizer = KBinsDiscretizer(n_bins = 5,encode="ordinal",strategy="kmeans")
# one_hot
onehot_encoder = OneHotEncoder(categories = "auto", drop = "first", handle_unknown='error',sparse = False)

In [15]:
base_info[auto_bin_features].shape

(12279, 140)

In [16]:
len(auto_bin_features)+len(categrical_features)

382

In [17]:
# Discretizer.fit(base_info[auto_bin_features])
# bin_categrical_features = Discretizer.transform(base_info[auto_bin_features])
# bin_categrical_features = pd.DataFrame(bin_categrical_features, columns = bin_features_head)

In [18]:
manual_bin = False
if manual_bin:
    all_cate_features = pd.concat([base_info[categrical_features], bin_categrical_features], axis = 1)
else:
    all_cate_features = base_info[categrical_features]

In [19]:
onehot_encoder = OneHotEncoder(categories = "auto", drop = "first", handle_unknown='error',sparse = False)

In [20]:
all_cate_features.dtypes

web_dum                       int64
ENTTYPE                       int64
ENTTYPE                       int64
INDUSTRYPHY                   int64
INDUSTRYPHY                   int64
                              ...  
13_income_tax_adjuste_isna    int64
13_income_taxable_isna        int64
13_incometax_taxable_isna     int64
13_incometax_amount_isna      int64
13_incometax_actual_isna      int64
Length: 244, dtype: object

In [21]:
onehot_encoder.fit(all_cate_features)

OneHotEncoder(drop='first', sparse=False)

In [22]:
one_hot_features = onehot_encoder.transform(all_cate_features)

In [23]:
one_hot_features.shape

(12279, 393)

In [24]:
train = pd.read_csv("./trainning_data/label_risk_company.csv")
train_id = set(train.entid)

train_id_0 = set(train[train["CaseType"] == 0].entid)
train_id_1 = set(train[train["CaseType"] == 1].entid)
train_id_2 = set(train[train["CaseType"] == 2].entid)
train_id_3 = set(train[train["CaseType"] == 3].entid)

In [25]:
train.shape[0]-len(set(train.entid))

29

In [26]:
test =  pd.read_csv("./test_data/test_data.csv")
test.shape

(1392, 1)

In [27]:
test_id = set(test.entid)

In [28]:
Ys = np.zeros(len(base_info.entid))
Ys = Ys - 1

In [29]:
Ys[base_info[base_info.entid.isin(train_id_0)].index] = 0
Ys[base_info[base_info.entid.isin(train_id_1)].index] = 1
Ys[base_info[base_info.entid.isin(train_id_2)].index] = 2
Ys[base_info[base_info.entid.isin(train_id_3)].index] = 3

In [30]:
test_index = base_info[base_info.entid.isin(test_id)].index
for i in range(len(Ys)):
    if Ys[i] == -1:
        if not i in test_index:
            Ys[i] = 4
            
for i in range(len(Ys)):
    if i in test_index:
        Ys[i] = -1            


In [31]:
Ys = pd.DataFrame(Ys)

In [32]:
#Ys.to_csv('label_all.csv')

In [33]:
test_index = Ys[Ys[0] == -1].index

In [34]:
train_index = Ys[Ys[0] != -1].index

In [35]:
one_hot_features.shape

(12279, 393)

In [36]:
base_info[auto_bin_features].values.shape

(12279, 140)

In [37]:
if manual_bin:
    train_x = one_hot_features[train_index,:]
else:
    train_x = np.concatenate([one_hot_features,base_info[auto_bin_features].values],axis = 1)[train_index,:]
train_y = Ys.iloc[train_index]

In [38]:
#OverSampler = RandomOverSampler()
OverSampler = SMOTE(random_state = 0)

In [39]:
#model
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn import svm

from sklearn.linear_model import SGDClassifier
# sgd_clf = SGDClassifier(loss = "hinge",
#                         penalty = "L2",
#                         random_state=42,
#                         verbose=1,
#                         max_iter=50)


#clf = RandomForestClassifier(max_depth=15, random_state=0)

import lightgbm as lgb
import xgboost
# model = lgb.LGBMClassifier(num_leaves=64,
#                            max_depth=10,
#                            learning_rate=0.01,
#                            n_estimators=10000,
#                            subsample=0.8,
#                            feature_fraction=0.8,
#                            reg_alpha=0.5,
#                            reg_lambda=0.5,
#                            random_state=42,
#                            n_jobs=40,
#                            metric=None)



#import xgboost as xgb
#model =xgb.XGBClassifier()


In [40]:
params = {'objective': 'multi:softprob', 
          'eval_metric': 'auc',
          'eta': 0.001,
          'max_depth': 10, 
          'subsample': 0.6, 
          'sampling_method ':"uniform",  #"uniform":  "gradient_based"
          'colsample_bytree': 0.6,
          'lambda':0.001, #L2
          'alpha':0.001,  #L1
          'random_state': 42,
          'num_class' : 5, 
          'silent': True}

params['gpu_id'] = 0
params['tree_method'] = 'gpu_hist'

In [ ]:
lr = 0.01
colsample_by_trees = [0.7, 0.8, 0.9, 1]
subsamples = [0.7, 0.8, 0.9, 1]
max_depths = [5, 10, 15, 20]
fold = 10


for colsample_by_tree,subsample,max_depth in product(colsample_by_trees,subsamples,max_depths):
    f1s = []
    for i in range(fold):
        X_train, X_eval, y_train, y_eval = train_test_split(train_x, train_y[0], test_size=0.2, random_state = i)
        train_x_oversampled, train_y_oversampled = OverSampler.fit_resample(X_train, y_train)

        #clf.fit(train_x_oversampled, train_y_oversampled)
    #     model = 
        #lgb

    #     clf = model.fit(train_x_oversampled,
    #                     train_y_oversampled,
    #                     eval_names=['valid'],
    #                     eval_set=[(X_eval, y_eval)],
    #                     verbose=0,
    #                     eval_metric='logloss',
    #                     early_stopping_rounds=50
    #                     )
        #xgb
        clf = xgboost.XGBClassifier(n_estimators = 100,
                                    max_depth = max_depth,
                                    use_label_encoder=False,
                                    learning_rate = lr,
                                    verbosity= 0,
                                    objective = "multi:softprob",
                                    booster = "gbtree",
                                    n_jobs = 10,
                                    reg_alpha = 0.0001,
                                    reg_lambda = 0.0001,
                                    random_state = 0, 
                                    num_parallel_tree = 5,
                                    colsample_by_tree = colsample_by_tree,
                                    subsample = subsample, 
                                    gpu_id = 3,
                                    tree_method = "gpu_hist")

        clf.fit(train_x_oversampled, train_y_oversampled)
        y_eval_pred = clf.predict(X_eval)
        f1 = f1_score(y_eval,y_eval_pred, average='macro')
#         acc = accuracy_score(y_eval, y_eval_pred)


        f1s.append(f1)

#       print("acc:{:.2f}, f1:{:.2f}".format(acc,f1))
#       target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']
        #print(classification_report(y_eval, y_eval_pred, target_names=target_names))
    
    avg_f1 = np.mean(f1s)
    std_f1 = np.std(f1s)
    print(colsample_by_tree,subsample,max_depth,avg_f1,std_f1)
    